In [ ]:
# Imports
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

ROOT = ''

In [ ]:
# Functions definitions
def read_trapezoidal():
    df = pd.read_csv(ROOT + 'data/external/trapezoidal_kc.csv',
                     sep=';', decimal=',',
                     index_col=0,
                     parse_dates=True, 
                     infer_datetime_format=True, dayfirst=True,
                     skiprows=[0],
                     header = 0,
                     names=['Allen', 'Rallo'],
    )
    df.index.name = 'Day'
    return df

def read_measures():
    df = pd.read_pickle(ROOT + 'data/interim/data.pickle')
    return df.loc[:, ['ETa', 'ETo']].dropna()

def read_kcvi():
    df = pd.read_csv(ROOT + 'data/external/VIs_Kc_2018_2022.csv',
                     sep=';', decimal=',',
                     index_col=0,
                     parse_dates=True, 
                     infer_datetime_format=True, dayfirst=True,
                     header = 0,
                     names = ['Kc_VIs']
    )
    df.index.name = 'Day'
    return df

def read_all_data():
    predicted_kc = pd.read_csv(ROOT + 'data/predicted/kc_postprocessed.csv', parse_dates=True, infer_datetime_format=True, index_col=0)
    predicted_trapezoidal = pd.read_pickle(ROOT + 'data/predicted/trapezoidal.pickle')
    theoretical = read_trapezoidal()
    kcvi = read_kcvi()
    return [predicted_kc, predicted_trapezoidal, theoretical, kcvi]

# Crop Coefficient Data

In [ ]:
import os
os.getcwd()

In [ ]:
data = pd.concat(read_all_data(), axis=1, join='outer')
data

## Measures and VI predictions
A comparison

In [ ]:
kc_measured = data.loc[data['Source']=='Measured', ['Kc', 'Kc_VIs']]
r2 = r2_score(kc_measured['Kc'], kc_measured['Kc_VIs'])
rmse = mean_squared_error(kc_measured['Kc'], kc_measured['Kc_VIs'], squared=False)

et_measures = read_measures()
eta_vi = kc_measured['Kc_VIs'] * et_measures['ETo']
eta = et_measures.loc[eta_vi.dropna().index, 'ETa']
r2_eta = r2_score(eta, eta_vi.dropna())
print(f'R2 score ETa measured / ETa VI: {r2_eta}')

In [ ]:
# Regression Plot
fig, ax = plt.subplots(figsize=(6, 6))
ax = sns.regplot(
    kc_measured, 
    x='Kc', 
    y='Kc_VIs',
    robust=True,
    ci=80,
    line_kws={'label':f"R2: {r2:.4}\nRMSE: {rmse:.4}", 'color':"red"},
    ax=ax
)
ax.legend(loc='upper left')
ax.set_ylim(0.45, 1.4)
ax.set_xlim(0.45, 1.4)

In [ ]:
g = sns.relplot(
    data, 
    x='Day', 
    y='Kc', 
    hue='Source',
    aspect=3,
)
g.map(sns.lineplot, 'Day', 'Kc_VIs', color='green', alpha=0.4)
g.set_ylabels('Crop Coefficient').set(title='Kc Measured, Predicted and Inferred by VI')

In [ ]:
data

In [ ]:
(low_trpz.plot())
(high_trpz.plot())
data.dropna().plot()

In [ ]:
g = sns.FacetGrid(data.reset_index(),
                 aspect=4)
g.map(sns.lineplot, 'Day', 'trapezoidal', color='blue')
g.map(sns.lineplot, 'Day', 'Allen', color='red')
g.set(title='Trapezoidal Predictions vs Allen theoretical trapezoid').set_ylabels('Crop Coefficient')
low_trpz = data['trapezoidal'].dropna()-data['std'].dropna()
high_trpz = data['trapezoidal'].dropna()+data['std'].dropna()
g.ax.fill_between(data.dropna().index, low_trpz.loc[data.dropna().index], high_trpz.loc[data.dropna().index], alpha=0.2)

## Mean Squared Error
Average distance between Measured/Predicted data and theoretical Allen trapezoid or VI inferred

In [ ]:
mse_allen = mean_squared_error(data['Kc'], data['Allen'], squared=False)
mse_vi = mean_squared_error(data['Kc'], data['Kc_VIs'], squared=False)
print(f'Computed Mean Squared Error (ALLEN): {mse_allen}')
print(f'Computed Mean Squared Error (VI): {mse_vi}')

## Relative Error
Plotting the relative error of each point, 
given by the ratio of the measure and the error

In [ ]:
import numpy as np
#data['Allen_Errors'] = mse_allen / data['Kc']
#data['VI_Errors'] = mse_vi / data['Kc']
data['Allen_Errors'] = (data['Kc'].values - data['Allen'].values) / data['Kc'].values
data['VI_Errors'] = (data['Kc'].values - data['Kc_VIs'].values) / data['Kc'].values

### Allen Errors

In [ ]:

g = sns.relplot(
    data,
    x='Day',
    y='Allen_Errors',
    hue='Source',
    aspect=2.4,
)
g.set(title='Kc Relative Error w/ respect to Allen predictions').set_ylabels('Crop Coefficient')

### VI Errors

In [ ]:
g = sns.relplot(
    data,
    x='Day',
    y='VI_Errors',
    hue='Source',
    aspect=2.4,
)
g.set(title='Kc Relative Error w/ respect to VIs predictions').set_ylabels('Crop Coefficient')

## Distributions

In [ ]:
g = sns.jointplot(data, 
    x='Kc', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)

In [ ]:
g = sns.jointplot(data, 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8) 

In [ ]:
g = sns.jointplot(data.loc['2018'], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('2018')
g.fig.tight_layout()

In [ ]:
g = sns.jointplot(data.loc['2019'], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('2019')
g.fig.tight_layout()

In [ ]:
g = sns.jointplot(data.loc['2020'], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('2020')
g.fig.tight_layout()

In [ ]:
g = sns.jointplot(data.loc['2021'], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('2021')
g.fig.tight_layout()

In [ ]:
data['Allen_Errors'].plot(kind='hist')
plt.show()

### Seasons

In [ ]:
g = sns.jointplot(data.loc['2018-05':'2018-08'], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('data between May 2018 and August 2018')
g.fig.tight_layout()

In [ ]:
g = sns.jointplot(data.loc['2018-01':'2018-03'], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('data between January 2018 and March 2018')
g.fig.tight_layout()

In [ ]:
g = sns.jointplot(data.loc['2018-09':'2018-12'], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('data between November 2018 and December 2018')
g.fig.tight_layout()

In [ ]:
g = sns.jointplot(data[(data.index.month < 9) & (data.index.month > 4)], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('data between May and August')
g.fig.tight_layout()

In [ ]:
g = sns.jointplot(data[(data.index.month < 4) | (data.index.month > 8)], 
    x='Kc_VIs', 
    y='Kc',
    kind='hist',
    cbar=True
)
g.fig.set_figwidth(8)
g.fig.suptitle('data before April and after August')
g.fig.tight_layout()

In [ ]:
g = sns.jointplot(
    data, 
    x='Allen', 
    y='Kc',
    kind='hist',
    cbar=True,
)
g.fig.set_figwidth(8)

In [ ]:
g = sns.jointplot(data, 
              x='Allen', 
              y='trapezoidal', 
              cbar=True, 
              kind='hist',
)
g.fig.set_figwidth(8)

In [ ]:
features = [col for col in data.columns if col != 'std']
g = sns.PairGrid(data, hue='Source', height=2)
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.histplot)

In [ ]:
features = [col for col in data.columns if col not in ['std', 'Source']]
g = sns.PairGrid(data, height=2, vars=features)
g.map_diag(sns.kdeplot)
g.map_offdiag(sns.histplot)

## Errors Violin Plots

In [ ]:
df_errors = pd.read_csv(ROOT + 'data/external/df_total_Kc.csv',
                        sep=';', decimal='.',
                        index_col=0,)
df_errors

In [ ]:
df_to_plot = df_errors.melt(id_vars=['Kc_mlp'], value_vars=['Allen', 'Rallo', 'Kc_Vls', 'Kc_kNN'], var_name='Model', value_name='Kc')
display(df_to_plot)
sns.violinplot(data=df_to_plot, x='Model', y='Kc_mlp')
plt.title('Kc Error Distribution across models')

In [ ]:
import numpy as np
df_to_plot['Kc_error'] = (df_to_plot['Kc'] - df_to_plot['Kc_mlp']).abs()
df_to_plot['Kc_error_squared'] = np.sqrt(np.abs(df_to_plot['Kc']**2 - df_to_plot['Kc_mlp']**2))
sns.violinplot(data=df_to_plot, x='Model', y='Kc_error')

In [ ]:
df_to_plot['Errors Frequency'] = df_to_plot['Kc_error'] / df_to_plot['Kc_error'].sum()
sns.displot(data=df_to_plot, x="Errors Frequency", hue="Model", 
            kind="hist", stat='frequency', 
            bins=5, multiple="dodge", shrink=.8)

In [ ]:
df_to_plot